In [2]:
# compartilhado com sucesso no github em 16/3...

# rodou tirando o input em 16/3

# comecou a nao rodar e não achamos o erro em 15/3. nao tinhamos versao salva anterior

# dict = {"client_id": "7842fixcbrvvpj", "client_secret": "d03imv2Ix1AYtSwl", "redirect_uri": "http://localhost:8080" }
# with open('credentials.json',"w") as a:
#         json.dump(dict,a,indent=4)

# meu cliente id: 78q2w2knusmsnt
# meu secret: rJ6Zbyc6TsoXvGVH

# link da pagina - 15/3: http://localhost:8080/?code=AQR00KmKuye9x65LHtblKYljnPkIDjo5njP0-gXkGSC1DXaUFUoiaCLgtIgwl067dtNAPHk5GlowTYT869_oB6psXttNUze2BEiGD11UIp-UcN2SyyHH1582GbDMTMLyvBUo9BotUeMqWebcgC6tWK83UHZ3xDwuzohWZycJCL8qzIjDmEVkV33P_N7Fmyf2vhWdnmuf9OXV6tPvpso&state=vzrofobsqgyydrnswqvu
# esse funcionou em 16/3 - http://localhost:8080/?code=AQQkcrApJkZfb0rvYGyVfTWMxWBJpQZXiRaX2CMNNu_qJ8jHt5kELmHbPS5zmJ8wEoTLFZQbB1q994QsvVKt--2lSS9NFacRWfcTl13YsicAGLDj8xmdMntOOxKj6UmigBT6AHjmpcw4YgsHUGKRhwnWuoDL3EQWFrn7cjEagWiSSReLqtjQ5U8Yui9BWJBC0ZzmkM9wqJFb_1iNssY&state=euzycjzwpzioymvqpcuq

# access code do input eh do link acima, depois de CODE e antes do STATE:

import json
import random
import requests
import string

# dict = {"client_id": "78q2w2knusmsnt", "client_secret": "rJ6Zbyc6TsoXvGVH", "redirect_uri": "http://localhost:8080" }
# with open('credentials.json',"w") as a:
#        json.dump(dict,a,indent=4)

def read_creds(filename):
    '''LER A  FILE COM AS CREDENTIALS'''
    with open(filename) as f:
        credentials = json.load(f)
    return credentials
 
creds = read_creds('credentials.json')
client_id, client_secret = creds['client_id'], creds['client_secret']
redirect_uri = creds['redirect_uri']



def create_CSRF_token():
    '''
    This function generates a random string of letters.
    It is not required by the Linkedin API to use a CSRF token.
    However, it is recommended to protect against cross-site request forgery
    '''
    letters = string.ascii_lowercase
    token = ''.join(random.choice(letters) for i in range(20))
    return token
def open_url(url):
    '''
    Function to Open URL.
    Used to open the authorization link
    '''
    import webbrowser
    print(url)
    webbrowser.open(url)
api_url = 'https://www.linkedin.com/oauth/v2'
 
def authorize(api_url,client_id,client_secret,redirect_uri):
    '''
    Make a HTTP request to the authorization URL.
    It will open the authentication URL.
    Once authorized, it'll redirect to the redirect URI given.
    The page will look like an error. but it is not.
    You'll need to copy the redirected URL.
    '''
    # Request authentication URL
    csrf_token = create_CSRF_token()
    params = {
        'response_type': 'code',
        'client_id': client_id,
        'redirect_uri': redirect_uri,
        'state': csrf_token,
        'scope': 'r_liteprofile,r_emailaddress,w_member_social'
        }
 
    response = requests.get(f'{api_url}/authorization',params=params)
 
    print(f'''
    The Browser will open to ask you to authorize the credentials.\n
    Since we have not set up a server, you will get the error:\n
    This site can’t be reached. localhost refused to connect.\n
    This is normal.\n
    You need to copy the URL where you are being redirected to.\n
    ''')
 
    open_url(response.url)
    redirect_response = input()
    auth_code = parse_redirect_uri(redirect_response)
    return auth_code



def parse_redirect_uri(redirect_response):
    '''
    Parse redirect response into components.
    Extract the authorized token from the redirect uri.
    '''
    from urllib.parse import urlparse, parse_qs
 
    url = urlparse(redirect_response)
    url = parse_qs(url.query)
    print(url)
    return url['code'][0]



def save_token(filename,data):
    '''
    Write token to credentials file.
    '''
    data = json.dumps(data, indent = 4)
    with open(filename, 'w') as f:
        f.write(data)
 



def headers(access_token):
    '''
    Make the headers to attach to the API call.
    '''
    headers = {
    'Authorization': f'Bearer {access_token}',
    'cache-control': 'no-cache',
    'X-Restli-Protocol-Version': '2.0.0'
    }
    return headers
def auth(credentials):
    '''
    Run the Authentication.
    If the access token exists, it will use it to skip browser auth.
    If not, it will open the browser for you to authenticate.
    You will have to manually paste the redirect URI in the prompt.
    '''
    creds = read_creds(credentials)
    print(creds)
    client_id, client_secret = creds['client_id'], creds['client_secret']
    redirect_uri = creds['redirect_uri']
    api_url = 'https://www.linkedin.com/oauth/v2'
         
    if 'access_token' not in creds.keys():
        args = client_id,client_secret,redirect_uri
        auth_code = authorize(api_url,*args)
        access_token = refresh_token(auth_code,*args)
        creds.update({'access_token':access_token})
        save_token(credentials,creds)
    else:
        access_token = creds['access_token']
    return access_token
def refresh_token(auth_code,client_id,client_secret,redirect_uri):
    '''
    Exchange a Refresh Token for a New Access Token.
    '''
    access_token_url = 'https://www.linkedin.com/oauth/v2/accessToken'
 
    data = {
        'grant_type': 'authorization_code',
        'code': auth_code,
        'redirect_uri': redirect_uri,
        'client_id': client_id,
        'client_secret': client_secret
        }
 
    response = requests.post(access_token_url, data=data, timeout=30)
    response = response.json()
    print(response)
    access_token = response['access_token']
    return access_token
if __name__ == '__main__':
    credentials = 'credentials.json'
    access_token = auth(credentials)
    # # Get the authorization verifier code from the callback url
    # redirect_response = input('Paste the full redirect URL here:')
    # auth_code = parse_redirect_uri(redirect_response)
    # return auth_code




#     with open(credentials.json) as f:
#         credentials = json.load(f)
#     return credentials
 
# creds = read_creds('credentials.json')
# client_id, client_secret = creds['client_id'], creds['client_secret']
# redirect_uri = creds['redirect_uri']


{'client_id': '78q2w2knusmsnt', 'client_secret': 'rJ6Zbyc6TsoXvGVH', 'redirect_uri': 'http://localhost:8080'}

    The Browser will open to ask you to authorize the credentials.

    Since we have not set up a server, you will get the error:

    This site can’t be reached. localhost refused to connect.

    This is normal.

    You need to copy the URL where you are being redirected to.

    
https://www.linkedin.com/uas/login?session_redirect=%2Foauth%2Fv2%2Flogin-success%3Fapp_id%3D209503230%26auth_type%3DAC%26flow%3D%257B%2522state%2522%253A%2522ilkknelbrlfvsmfzjzgt%2522%252C%2522scope%2522%253A%2522r_liteprofile%252Cr_emailaddress%252Cw_member_social%2522%252C%2522appId%2522%253A209503230%252C%2522authorizationType%2522%253A%2522OAUTH2_AUTHORIZATION_CODE%2522%252C%2522authFlowName%2522%253A%2522generic-permission-list%2522%252C%2522currentStage%2522%253A%2522LOGIN_SUCCESS%2522%252C%2522currentSubStage%2522%253A0%252C%2522creationTime%2522%253A1678971927434%252C%2522redirectUri%252